# 20220510-zoo-table
- Collecting statistics (from `Pythia` directory):
    - Create Zoo sweep by running `./experiments/prefetcher_zoo.py condor_setup`
    - Evaluate Zoo sweep by running `./experiments/prefetcher_zoo.py eval`

In [52]:
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import display

from utils import utils, stats

from importlib import reload
reload(stats)
reload(utils)

<module 'utils.utils' from '/u/cmolder/GitHub/prefetching-zoo/utils/utils.py'>

In [53]:
weight_path = '/scratch/cluster/cmolder/traces/spec06/champsim/weights.txt'
weights = utils.read_weights_file(weight_path)

### Add SPEC results

In [54]:
# Individual prefetchers
path_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/individual.csv'
df = utils.read_data_file(path_spec)

# Online PC-localized
# pta       :
# ptmu      :
# ptnu      : num_useful, invoke PC prefetchers only, register fill to all prefetchers
# ptnu_cred : num_useful, invoke all prefetchers, register fill to all prefetchers that prefetched address
path_online_pc_accuracy_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/online_pc_accuracy.csv'
path_online_pc_num_useful_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/online_pc_num_useful.csv'
path_online_pc_marg_useful_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/online_pc_marginal_useful.csv'
path_online_pc_num_useful_all_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/online_pc_num_useful_all.csv'
path_online_pc_num_useful_cred_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/online_pc_num_useful_credit.csv'

df_on_pta = utils.read_data_file(path_online_pc_accuracy_spec)
df_on_pta.LLC_pref = 'online_pc_acc'
df_on_pta.all_pref = [('no', 'no', 'online_pc_acc')]*len(df_on_pta)
df = df.append(df_on_pta)

df_on_ptnu = utils.read_data_file(path_online_pc_num_useful_spec)
df_on_ptnu.LLC_pref = 'online_pc_num_useful'
df_on_ptnu.all_pref = [('no', 'no', 'online_pc_num_useful')]*len(df_on_ptnu)
df = df.append(df_on_ptnu)

df_on_ptmu = utils.read_data_file(path_online_pc_marg_useful_spec)
df_on_ptmu.LLC_pref = 'online_pc_marg_useful'
df_on_ptmu.all_pref = [('no', 'no', 'online_pc_marg_useful')]*len(df_on_ptmu)
df = df.append(df_on_ptmu)

df_on_ptnu_all = utils.read_data_file(path_online_pc_num_useful_all_spec)
df_on_ptnu_all.LLC_pref = 'online_pc_num_useful_all'
df_on_ptnu_all.all_pref = [('no', 'no', 'online_pc_num_useful_all')]*len(df_on_ptnu_all)
df = df.append(df_on_ptnu_all)

df_on_ptnu_cred = utils.read_data_file(path_online_pc_num_useful_cred_spec)
df_on_ptnu_cred.LLC_pref = 'online_pc_num_useful_credit'
df_on_ptnu_cred.all_pref = [('no', 'no', 'online_pc_num_useful_credit')]*len(df_on_ptnu_cred)
df = df.append(df_on_ptnu_cred)

# Offline PC-localized
path_offline_pc_accuracy_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/offline_pc_accuracy.csv'
path_offline_pc_num_useful_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/offline_pc_num_useful.csv'
path_offline_pc_marg_useful_spec = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_spec06/offline_pc_marginal_useful.csv'

df_off_pta = utils.read_data_file(path_offline_pc_accuracy_spec)
df_off_pta.LLC_pref = 'offline_pc_acc'
df_off_pta.all_pref = [('no', 'no', 'offline_pc_acc')]*len(df_off_pta)
df = df.append(df_off_pta)

df_off_ptnu = utils.read_data_file(path_offline_pc_num_useful_spec)
df_off_ptnu.LLC_pref = 'offline_pc_num_useful'
df_off_ptnu.all_pref = [('no', 'no', 'offline_pc_num_useful')]*len(df_off_ptnu)
df = df.append(df_off_ptnu)

df_off_ptmu = utils.read_data_file(path_offline_pc_marg_useful_spec)
df_off_ptmu.LLC_pref = 'offline_pc_marg_useful'
df_off_ptmu.all_pref = [('no', 'no', 'offline_pc_marg_useful')]*len(df_off_ptmu)
df = df.append(df_off_ptmu)

### Add GAP results

In [55]:
# Individiual prefetchers
path_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/individual.csv'
df_gap = utils.read_data_file(path_gap)
df = df.append(df_gap)

# Online PC-localized
path_online_pc_accuracy_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/online_pc_accuracy.csv'
path_online_pc_num_useful_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/online_pc_num_useful.csv'
path_online_pc_marg_useful_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/online_pc_marginal_useful.csv'
path_online_pc_num_useful_all_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/online_pc_num_useful_all.csv'
path_online_pc_num_useful_cred_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/online_pc_num_useful_credit.csv'

df_on_pta = utils.read_data_file(path_online_pc_accuracy_gap)
df_on_pta.LLC_pref = 'online_pc_acc'
df_on_pta.all_pref = [('no', 'no', 'online_pc_acc')]*len(df_on_pta)
df = df.append(df_on_pta)

df_on_ptnu = utils.read_data_file(path_online_pc_num_useful_gap)
df_on_ptnu.LLC_pref = 'online_pc_num_useful'
df_on_ptnu.all_pref = [('no', 'no', 'online_pc_num_useful')]*len(df_on_ptnu)
df = df.append(df_on_ptnu)

df_on_ptmu = utils.read_data_file(path_online_pc_marg_useful_gap)
df_on_ptmu.LLC_pref = 'online_pc_marg_useful'
df_on_ptmu.all_pref = [('no', 'no', 'online_pc_marg_useful')]*len(df_on_ptmu)
df = df.append(df_on_ptmu)

df_on_ptnu_all = utils.read_data_file(path_online_pc_num_useful_all_gap)
df_on_ptnu_all.LLC_pref = 'online_pc_num_useful_all'
df_on_ptnu_all.all_pref = [('no', 'no', 'online_pc_num_useful_all')]*len(df_on_ptnu_all)
df = df.append(df_on_ptnu_all)

df_on_ptnu_cred = utils.read_data_file(path_online_pc_num_useful_cred_gap)
df_on_ptnu_cred.LLC_pref = 'online_pc_num_useful_credit'
df_on_ptnu_cred.all_pref = [('no', 'no', 'online_pc_num_useful_credit')]*len(df_on_ptnu_cred)
df = df.append(df_on_ptnu_cred)

# Offline PC-localized
path_offline_pc_accuracy_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/offline_pc_accuracy.csv'
path_offline_pc_num_useful_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/offline_pc_num_useful.csv'
# path_offline_pc_marg_useful_gap = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_25_gap/offline_pc_marginal_useful.csv'

df_off_pta = utils.read_data_file(path_offline_pc_accuracy_gap)
df_off_pta.LLC_pref = 'offline_pc_acc'
df_off_pta.all_pref = [('no', 'no', 'offline_pc_acc')]*len(df_off_pta)
df = df.append(df_off_pta)

df_off_ptnu = utils.read_data_file(path_offline_pc_num_useful_gap)
df_off_ptnu.LLC_pref = 'offline_pc_num_useful'
df_off_ptnu.all_pref = [('no', 'no', 'offline_pc_num_useful')]*len(df_off_ptnu)
df = df.append(df_off_ptnu)

df_off_ptmu = utils.read_data_file(path_online_pc_marg_useful_gap)
df_off_ptmu.LLC_pref = 'offline_pc_marg_useful'
df_off_ptmu.all_pref = [('no', 'no', 'offline_pc_marg_useful')]*len(df_off_ptmu)
df = df.append(df_off_ptmu)

### Add CloudSuite results

In [56]:
### Add CloudSuite results

# Individual prefetchers
path_cs = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo/2022_04_08_cloudsuite/individual.csv'
df_cs = utils.read_data_file(path_cs)
df = df.append(df_cs)

---
## Helper functions

In [57]:
all_opportunity_prefetchers = [
    ('no', 'no', 'offline_phase'), 
    ('no', 'no', 'online_pc_acc'), 
    ('no', 'no', 'online_pc_num_useful'),
    ('no', 'no', 'online_pc_marg_useful'),
    ('no', 'no', 'online_pc_num_useful_all'),
    ('no', 'no', 'online_pc_num_useful_credit'),
    ('no', 'no', 'offline_pc_acc'), 
    ('no', 'no', 'offline_pc_num_useful'),
    ('no', 'no', 'offline_pc_marg_useful'),
]

def mean_stats_table(df, weights, metrics=['ipc_improvement'], method='weighted',
                     prefetchers=None,
                     opportunity_prefetchers=all_opportunity_prefetchers,
                     benchmarks=None):
    
    # Filter benchmarks
    if benchmarks is not None:
        df = df[df.trace.isin(benchmarks)]
        
    # Filter or weight simpoints  
    # Add phase_combined
    if method == 'weighted':
        df = stats.get_weighted_statistics(df, weights, add_phase_combined=True) # Weighted simpoints
    else:
        df = df[df.full_trace.isin(stats.get_longest_simpoints(weights))] # Longest simpoint
        
    # Filter prefetchers by weighted simpoints
    if prefetchers is not None:
        df = df[df.all_pref.isin(prefetchers + opportunity_prefetchers)]
    
    # Create table
    means = defaultdict(list)
    for pref in df.all_pref.unique():
        data = df[df.all_pref == pref]
        means['prefetcher'].append(pref)
        for metric in metrics:
            means[metric].append(utils.mean(data[metric], metric))
    means_df = pd.DataFrame(means)
    display(means_df)

---
## SPEC '06 cumulative statistics

### Best static prefetcher vs. opportunity

In [58]:
spec_top_1 = utils.rank_prefetchers(df[df.trace.isin(utils.spec)], 'ipc_improvement', 1)
mean_stats_table(
    df, weights, 
    metrics=['LLC_accuracy', 'LLC_coverage', 'LLC_mpki_reduction', 'ipc_improvement'],
    method='weighted',
    prefetchers=spec_top_1, 
    opportunity_prefetchers=all_opportunity_prefetchers,
    benchmarks=utils.spec
)

,prefetcher,LLC_accuracy,LLC_coverage,LLC_mpki_reduction,ipc_improvement
0,"(no, no, pythia_sisb)",80.475007,56.733090,72.165253,34.697132
1,"(no, no, online_pc_acc)",91.873877,49.741285,66.433235,27.064810
2,"(no, no, online_pc_num_useful)",76.021616,58.542962,75.155191,33.433478
3,"(no, no, online_pc_marg_useful)",75.853307,58.591305,75.167900,33.628640
4,"(no, no, online_pc_num_useful_all)",75.712230,60.476507,76.376049,35.958425
5,"(no, no, online_pc_num_useful_credit)",76.006492,60.449208,76.343812,35.963601
6,"(no, no, offline_pc_acc)",92.673781,50.337269,66.650811,27.824458
7,"(no, no, offline_pc_num_useful)",76.097135,61.093447,76.585136,35.930998
8,"(no, no, offline_pc_marg_useful)",76.033296,61.090964,76.584459,35.932005
9,"(no, no, offline_phase)",92.516872,62.980567,78.032093,39.972902


### Opportunity prefetchers

---
## GAP cumulative statistics

### Top 5 by IPC improvement
- Note: Weighted vs. longest simpoint doesn't matter here, since all gap traces are single-simpoint.

In [59]:
gap_top_1 = utils.rank_prefetchers(df[df.trace.isin(utils.gap)], 'ipc_improvement', 1)
mean_stats_table(
    df, weights, 
    metrics=['LLC_accuracy', 'LLC_coverage', 'LLC_mpki_reduction', 'ipc_improvement'],
    method='weighted',
    prefetchers=gap_top_1, 
    opportunity_prefetchers=all_opportunity_prefetchers,
    benchmarks=utils.gap
)

,prefetcher,LLC_accuracy,LLC_coverage,LLC_mpki_reduction,ipc_improvement
0,"(no, no, bingo_sisb)",63.814534,84.950145,92.235936,19.103641
1,"(no, no, online_pc_acc)",89.230651,60.300438,63.179017,13.869738
2,"(no, no, online_pc_num_useful)",70.312822,80.721726,89.995176,17.603572
3,"(no, no, online_pc_marg_useful)",70.329205,80.647598,89.800834,17.522176
4,"(no, no, online_pc_num_useful_all)",70.378947,81.155454,89.946118,17.639879
5,"(no, no, online_pc_num_useful_credit)",70.385113,81.167499,89.942794,17.620523
6,"(no, no, offline_pc_acc)",89.703817,65.554118,71.046757,14.084112
7,"(no, no, offline_pc_num_useful)",70.595639,81.164110,89.868650,17.619107
8,"(no, no, offline_pc_marg_useful)",70.329205,80.647598,89.800834,17.522176
9,"(no, no, offline_phase)",89.094475,85.257951,92.996483,19.150158
